## CTA simulations

First we import the libraries.

In [1]:
# import the required libraries
import os
from os.path import expandvars
import ctools

We can make a folder to store the example data that will be created in this tutorial and used in the futher ones. 

In [2]:
if not os.path.isdir('../data'):
    os.mkdir('../data')

We need to export the two main environmental variables for ctools (if you already saved them in the notebook settings then you don't require this passage). You will simply have to change the path in order to provide the location of the <code>CTOOLS</code> and <code>CALDB</code> folders. See below:

In [3]:
os.environ["CALDB"] = expandvars("$CTOOLS/share/caldb/")
print(expandvars("$CTOOLS"))
print(expandvars("$CALDB"))

/data01/homes/dipiano/.conda/envs/cta_simulation
/data01/homes/dipiano/.conda/envs/cta_simulation/share/caldb/


For simplicity let's make a wrapper of the ctools MC simulator. We can write a function with some default parameters in order to invoke it when needed.

In [4]:
def run_ctobssim(model, pointing, output, energy=(0.03, 150), time=(0, 1200), fov=5, caldb='prod3b-v2', irf='South_z40_0.5h', seed=1):
    sim = ctools.ctobssim()
    sim["inmodel"] = model
    sim["outevents"] = output
    sim["caldb"] = caldb
    sim["irf"] = irf
    sim["ra"] = pointing[0]
    sim["dec"] = pointing[1]
    sim["rad"] = fov
    sim["tmin"] = time[0]
    sim["tmax"] = time[1]
    sim["emin"] = energy[0]
    sim["emax"] = energy[1]
    sim["seed"] = seed
    sim.execute()
    # if you want to run in memory:
    #sim.run()

The same thing we do with the tool that produces the skymaps, we make a wrapping function to call at our need.

In [5]:
def run_skymap(obs, output, energy=(0.03, 150), roi=5, caldb='prod3b-v2', irf='South_z40_0.5h', wbin=0.02):
    nbin = int(roi*2/wbin)
    skymap = ctools.ctskymap()
    skymap['inobs'] = obs
    skymap['outmap'] = output
    skymap['irf'] = irf
    skymap['caldb'] = caldb
    skymap['emin'] = energy[0]
    skymap['emax'] = energy[1]
    skymap['usepnt'] = True
    skymap['nxpix'] = nbin
    skymap['nypix'] = nbin
    skymap['binsz'] = wbin
    skymap['bkgsubtract'] = 'IRF'
    skymap.execute()
    # if you want to run in memory:
    #sim.run()

Let's fix the pointing.

In [6]:
pointing = (83.6331, 22.5145)

## Background simulation and map

We can run the simulation of the background and then use the observation produced to run create the skymap. The simulation will be used in tutorial <code>3_find_prefactor.ipynb</code> to evaluate the background level and find the prefactor required to simulate a source with given significance. The skymap can be used as visual inspection of the simulation and as part of the training of the ML algorithm.

In [7]:
model = '$CTOOLS/share/models/bkg_irf.xml'
run_ctobssim(model=model, pointing=pointing, output='../data/bkg_test_sim.fits')
obs = '../data/bkg_test_sim.fits'
run_skymap(obs=obs, output='../data/bkg_test_sky.fits')

## Source simulation and map

Before this step you should have modified the source model (i.e., crab.xml) with the correct prefactor. You can do so manually or use the provided class in <code>sagsci</code> package for which you can find an example in <code>4_update_source_model.ipynb</code>.

In [8]:
model = '$CTOOLS/share/models/crab.xml'
run_ctobssim(model=model, pointing=pointing, output='../data/crab_test_sim.fits')
obs = '../data/crab_test_sim.fits'
run_skymap(obs=obs, output='../data/crab_test_sky.fits')